In [35]:
with open('./aclImdb/imdb.vocab') as f:
    vocabulary = [word.rstrip() for word in f]
    vocabulary = vocabulary[:10000] # keeping most frequent 10k to save memory
    print('%d words in vocabulary' % (len(vocabulary),))
import re 
import nltk 
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
stop_words=set(stopwords.words('english'))


10000 words in vocabulary


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/pankajgope/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pankajgope/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/pankajgope/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
#Generating Tokens
def generate_tokens(text):
    text = text.lower()
    text = re.sub("\\s", " ", text) 
    text = re.sub("[^a-z' ]", "", text) 
    token_words = word_tokenize(text)
    tokens = []
    for i in token_words:
        if i not in stop_words:
            tokens.append(i)

    tokens_last=[]

    # Lemmatization 
    lemmatizer = WordNetLemmatizer()
    for j in tokens:
      tokens_last.append(lemmatizer.lemmatize(j))

    print("Final Tokens Output: ")
    print(tokens_last)

    return tokens

In [31]:
#Loading the dataset
import os
def load_dataset(dirname):
    x, y = [], []
    
    for yvalue, ylabel in enumerate(['neg', 'pos']):
        ydirectory = os.path.join(dirname, ylabel)
        for filename in os.listdir(ydirectory):
            filepath = os.path.join(ydirectory, filename)
            with open(filepath) as f:
                tokens = generate_tokens(f.read())
            x.append(tokens)
            y.append(yvalue)
    print()
    return x, y

In [34]:
x_train, y_train = load_dataset('./aclImdb')

FileNotFoundError: [Errno 2] No such file or directory: './aclImdb/neg'

In [33]:
X_train, y_train = load_dataset('./aclImdb/tr')

X_test, y_test = load_dataset('./aclImdb/tes')

FileNotFoundError: [Errno 2] No such file or directory: './aclImdb/tr/neg'

In [ ]:
#training data length
len(X_train), len(y_train)

In [ ]:
#testing data length
len(X_test), len(y_test)

In [ ]:
def bow_onehot_vector(tokens):
    vector = [0] * len(vocabulary)
    for t in tokens:
        try:
            vector[vocabulary.index(t)] = 1
        except:
            pass  # ignore missing words
    return vector

In [ ]:
from tqdm import tqdm #progress bar

X_bow_train = [bow_onehot_vector(x) for x in tqdm(X_train)]
X_bow_val = [bow_onehot_vector(x) for x in tqdm(X_test)]

In [ ]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential #Sequential API is used to create models' layer-by-layer.
from keras.layers import Dense, LSTM, Dropout #Dense layer is the regular deeply connected neural network layer
from keras.layers.convolutional import Conv1D, MaxPooling1D #1D convolution layer , Max pooling operation for 1D temporal data.
from keras.layers.embeddings import Embedding #Embedding layer enables us to convert each word into a fixed length vector of defined size.
from keras.preprocessing import sequence
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#fix random seed for reproducibility
numpy.random.seed(5)

In [ ]:
# load the dataset but only keep the top n words, zero the rest
top_words = 4000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [ ]:
# truncate and pad input sequences
max_review_length = 600
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [ ]:
# create the model
embedding_vecor_length = 64 #Embedding layer enables us to convert each word into a fixed length vector of defined size.
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length)) #maps a sequence of word.
model.add(Conv1D(filters=32, kernel_size=4, padding='same', activation='relu')) #relu "The function returns 0 if it receives any negative input".
model.add(MaxPooling1D(pool_size=3)) #MaxPooling It will take highest vale from each matrix and stores in one matrix.
model.add(Dropout(0.2)) #used to prevent a model from overfitting.
model.add(LSTM(200, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
print(model.summary())

In [ ]:
model.fit(X_train, y_train, epochs=8, batch_size=64)

In [ ]:
# Final evaluation of the model
scores_1 = b.evaluate(X_train, y_train, verbose=0)
print("Training Accuracy: %.2f%%" % (scores_1[1]*100))

scores_2 = b.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: %.2f%%" % (scores_2[1]*100))